# CIMantic Graphs Overview

This tutorial provides an introduction to usage of the CIMantic Graphs library (aka CIM-Graph). 

CIMantic Graphs is an open-source library for creating in-memory labeled property graphs for creating, parsing, and editing CIM power system models. It creates Python object instances in memory using a data profile exported from a specified CIM profile (e.g. IEC61970cim17v40 or GridAPPS-D RC4_2021).

To install CIMantic Graphs clone the github repository or use pip install: `pip install cim-graph`

## Table of Contents:

* [1. Importing CIMantif Graphs]()
* [2. Specifying Connection Parameters](#2-specifying-connection-parameters)
* [3. Connecting to the Data Source]()


----

## 1. Importing CIMantic Graphs

CIMantic Graphs contains several packages within the core library:
* `data_profiles`: This package contains full CIM profiles exported through CIMTool or Enterprise Architect Schema Composer.
* `databases`: This package contains database i/o connections and backend query handling for multiple databases (e.g. Blazegraph, GraphDB, Neo4J, MySQL, etc.)
* `models`: This package contains knowledge graph models for transmission node-breaker, transmission bus-branch, and distribution feeder models.
* `queries`: This package contains generic queries that are built at runtime for any CIM profile and CIM class

### 1.1. Importing the CIM Profile
The first step in using CIMantic Graphs is to import the python data profile for desired CIM profile. The data profile provides the ability to invoke CIM classes directly based on their name.



__Method 1:__ Directly import the desired CIM profile:

In [1]:
import cimgraph.data_profile.rc4_2021 as cim

__Method 2 (recommended):__ Use `importlib`:

In [2]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

After importing the data profile, it is possible to create an instance of a class or view the attributes of any class. 

__Example 1:__ Create a new breaker with a name and mRID. The `uuid` library can be used to generate the unique identifier used for all CIM objects.

In [3]:
import uuid
breaker = cim.Breaker(name = "breaker_01", mRID = str(uuid.uuid4()))
breaker.open = True
print(breaker)

Breaker(mRID='f6a5250e-d6ae-4ee0-a760-4bce56950ad2', aliasName=None, description=None, name='breaker_01', Names=[], AssetDatasheet=None, Assets=[], ConfigurationEvent=[], Controls=[], Location=None, Measurements=[], OperatingShare=[], PSRType=None, ReportingGroup=[], aggregate=None, inService=None, networkAnalysisEnabled=None, normallyInService=None, AdditionalEquipmentContainer=[], EquipmentContainer=None, Faults=[], OperationalLimitSet=[], UsagePoints=None, BaseVoltage=None, SvStatus=[], Terminals=[], normalOpen=None, open=True, ratedCurrent=None, retained=None, CompositeSwitch=None, SvSwitch=[], SwitchPhase=[], SwitchSchedules=[], breakingCapacity=None, inTransitTime=None)


__Example 2:__ Associate two CIM objects based on their associations

In [4]:
substation = cim.Substation(name = "sub_1", mRID = str(uuid.uuid4()))
breaker.EquipmentContainer = substation

__Example 3:__ View documentation of the ACLineSegment class

In [5]:
print(cim.ACLineSegment.__doc__)

A wire or combination of wires, with consistent electrical
    characteristics, building a single electrical system, used to carry
    alternating current between points in the power system.

    For symmetrical, transposed 3ph lines, it is sufficient to use
    attributes of the line segment, which describe impedances and
    admittances for the entire length of the segment.  Additionally
    impedances can be computed by using length and associated per length
    impedances. The BaseVoltage at the two ends of ACLineSegments in a
    Line shall have the same BaseVoltage.nominalVoltage. However,
    boundary lines  may have slightly different
    BaseVoltage.nominalVoltages and  variation is allowed. Larger
    voltage difference in general requires use of an equivalent branch.

    :ivar b0ch: Zero sequence shunt (charging) susceptance, uniformly
        distributed, of the entire line section.
    :ivar bch: Positive sequence shunt (charging) susceptance, uniformly
        distribute

-----

## 2. Specifying Connection Parameters

The next step in using any of the CIMantic Graphs library classes to establish the connection parameters for reading or writing the CIM model. The `ConnectionParameters` class can be imported through:

In [6]:
from cimgraph import ConnectionParameters

The parameters can be defined using an instance specifying the required paramters for the specific connection interface (database) to be used.

In [7]:
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
                              cim_profile='rc4_2021', iec61970_301=7) # Blazegraph params

The required and optional arguments for the `ConnectionParameters` class are described in detail in [ConnectionParameters Arguments](link)

----

## 3. Connecting to the Data Source

The next step is to connect to the database or file source that will be used for CIM model. A complete list of connection types currently supported are described in [Supported Databases](link)

In [8]:
from cimgraph.databases.blazegraph import BlazegraphConnection
blazegraph = BlazegraphConnection(params)

----

## 4. Creating a Container and Graph Model

CIMantic Graphs uses an EquipmentContainer class as the starting point for building a knowledge graph of the power system model. The supported classes are `BusBranchModel`, `NodeBreakerModel`, and `FeederModel`.

In [10]:
from cimgraph.models import FeederModel

The power system network is then created by passing the database connection, container object, and a boolean flag whether a centralized or distributed model should be built.

In [12]:
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
feeder = cim.Feeder(mRID=feeder_mrid)
network = FeederModel(connection=blazegraph, container=feeder, distributed=False)

The network is populated by default with all ConductingEquipment, ConnectivityNode, and Terminal class instances. The knowledge graph is indexed by the class type and then the device mRID.


To view instances of a particular class, use the `.pprint(cim.ClassName)` method. Use of the default python print method is not recommended due to foward-reverse relationships resulting in an infinite print loop.

In [13]:
network.pprint(cim.Breaker)

{
    "_52DE9189-20DC-4C73-BDEE-E960FE1F9493": {
        "mRID": "_52DE9189-20DC-4C73-BDEE-E960FE1F9493",
        "Terminals": [
            "_1D81C7FE-E88F-41E3-A900-476CA6476CCD",
            "_2847E06B-C8ED-41E6-B515-C61C9E8EB4B4"
        ]
    }
}


----

## 5. Automated Database Queries
CIMantic Graphs contains automatic query generation routines for all classes and all supported databases using the `.get_all_edges(cim.ClassName)` method. This query obtains all attributes for all objects of that class type and expands the knowledge graph by one edge with default instances of all associated objects.

In [25]:
network.get_all_edges(cim.Breaker)
network.get_all_edges(cim.Terminal)
network.get_all_edges(cim.ConnectivityNode)

In [19]:
network.pprint(cim.Breaker)

{
    "_52DE9189-20DC-4C73-BDEE-E960FE1F9493": {
        "mRID": "_52DE9189-20DC-4C73-BDEE-E960FE1F9493",
        "name": "brkr1",
        "Location": "_085BBE1F-FF95-4260-A9D2-8D2F1A8EA9A3",
        "Measurements": [
            "_40ac2899-1d2a-469f-a14a-1e14ea29d172",
            "_7c6f94c1-1419-4582-ab2d-a0b11772c26b",
            "_8e7f04ee-a032-4128-838e-a3442a1c3026",
            "_ab18a8e1-f023-4f9e-bf02-c75bf05164df",
            "_b393e719-0981-4498-9d96-07f1be7de009",
            "_f11a9ad9-5b68-483b-b52f-dd4af07bb77d",
            "_0c48c74a-ceee-4c99-bd73-28079561ca7a",
            "_3c60208a-8ef8-483c-828b-30ee42d402dc",
            "_43f80e34-281e-4baa-8aba-d931a9a3ab87",
            "_9f5cb9c4-71d6-4f2b-9dc1-26c7e9f84410",
            "_aec42b89-f3c0-47e9-b21a-82736b2a1149",
            "_baccfd49-ec98-4380-8be9-d242dc8611f3"
        ],
        "EquipmentContainer": "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "BaseVoltage": "_2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
 

---

## 6. Traversing the Knowledge Graph
CIMantic Graphs associates CIM classes by creating direct references between in-memory object instances based on the naming and hierarchy of the underlying information model. 

To view the attributes of particular object instance, directly invoke the attribute from the UML class diagram.

In [31]:
breaker = network.graph[cim.Breaker]["_52DE9189-20DC-4C73-BDEE-E960FE1F9493"]
print(breaker.normalOpen)

false


To traverse the knowledge graph, no custom queries are required. Instead, directly invoke the UML association names that serves as references between objects in the graph:

In [30]:
print(breaker.Terminals[0].ConnectivityNode.name)

650


No separate queries or mapping are required for measurment objects. Call the `.get_all_edges` method for each measurement class, and then obtain the measurements from the equipment object

In [33]:
network.get_all_edges(cim.Analog)
network.get_all_edges(cim.Discrete)

In [36]:
for meas in breaker.Measurements:
    print(meas.name, meas.phases, meas.Terminal.ConnectivityNode.name)

Breaker_brkr1_Voltage PhaseCode.A 650
Breaker_brkr1_Voltage PhaseCode.B brkr
Breaker_brkr1_Voltage PhaseCode.B 650
Breaker_brkr1_Voltage PhaseCode.C 650
Breaker_brkr1_Voltage PhaseCode.C brkr
Breaker_brkr1_Voltage PhaseCode.A brkr
Breaker_brkr1_Current PhaseCode.A 650
Breaker_brkr1_Current PhaseCode.B 650
Breaker_brkr1_Current PhaseCode.C 650
Breaker_brkr1_State PhaseCode.A 650
Breaker_brkr1_State PhaseCode.C 650
Breaker_brkr1_State PhaseCode.B 650
